https://wikidocs.net/64765

## 훈련 데이터 전처리

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
sentence = "Repeat is the best medicine for memory".split()

In [3]:
vocab = list(set(sentence))
print(vocab)

['medicine', 'Repeat', 'memory', 'best', 'is', 'the', 'for']


단어장의 단어에 고유한 정수 인덱스를 부여

In [4]:
word2index = {tkn: i for i, tkn in enumerate(vocab, 1)}  # 단어에 고유한 정수 부여
word2index['<unk>']=0
print(word2index)

{'medicine': 1, 'Repeat': 2, 'memory': 3, 'best': 4, 'is': 5, 'the': 6, 'for': 7, '<unk>': 0}


수치화된 데이터를 단어로 바꾸기 위한 사전

In [5]:
index2word = {v: k for k, v in word2index.items()}
print(index2word)

{1: 'medicine', 2: 'Repeat', 3: 'memory', 4: 'best', 5: 'is', 6: 'the', 7: 'for', 0: '<unk>'}


데이터의 각 단어를 정수로 인코딩하는 동시에, 입력 데이터와 레이블 데이터를 만드는 build_data라는 함수

In [6]:
def build_data(sentence, word2index):
    encoded = [word2index[token] for token in sentence] # 각 문자를 정수로 변환. 
    input_seq, label_seq = encoded[:-1], encoded[1:] # 입력 시퀀스와 레이블 시퀀스를 분리
    input_seq = torch.LongTensor(input_seq).unsqueeze(0) # 배치 차원 추가
    label_seq = torch.LongTensor(label_seq).unsqueeze(0) # 배치 차원 추가
    return input_seq, label_seq

입력 데이터와 레이블 데이터를 얻습니다.

In [7]:
X, Y = build_data(sentence, word2index)
print(X)
print(Y)

tensor([[2, 5, 6, 4, 1, 7]])
tensor([[5, 6, 4, 1, 7, 3]])


모델들과 달라진 점은 임베딩 층을 추가했다는 점
파이토치에서는 nn.Embedding()을 사용해서 임베딩 층을 구현
- 첫번째 인자는 단어장의 크기이며
- 두번째 인자는 임베딩 벡터의 차원

In [8]:
class Net(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size, batch_first=True):
        super(Net, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, # 워드 임베딩
                                            embedding_dim=input_size)
        self.rnn_layer = nn.RNN(input_size, hidden_size, # 입력 차원, 은닉 상태의 크기 정의
                                batch_first=batch_first)
        self.linear = nn.Linear(hidden_size, vocab_size) # 출력은 원-핫 벡터의 크기를 가져야함. 또는 단어 집합의 크기만큼 가져야함.

    def forward(self, x):
        # 1. 임베딩 층
        # 크기변화: (배치 크기, 시퀀스 길이) => (배치 크기, 시퀀스 길이, 임베딩 차원)
        output = self.embedding_layer(x)
        # 2. RNN 층
        # 크기변화: (배치 크기, 시퀀스 길이, 임베딩 차원)
        # => output (배치 크기, 시퀀스 길이, 은닉층 크기), hidden (1, 배치 크기, 은닉층 크기)
        output, hidden = self.rnn_layer(output)
        # 3. 최종 출력층
        # 크기변화: (배치 크기, 시퀀스 길이, 은닉층 크기) => (배치 크기, 시퀀스 길이, 단어장 크기)
        output = self.linear(output)
        # 4. view를 통해서 배치 차원 제거
        # 크기변화: (배치 크기, 시퀀스 길이, 단어장 크기) => (배치 크기*시퀀스 길이, 단어장 크기)
        return output.view(-1, output.size(2))

In [9]:
# 하이퍼 파라미터
vocab_size = len(word2index)  # 단어장의 크기는 임베딩 층, 최종 출력층에 사용된다. <unk> 토큰을 크기에 포함한다.
input_size = 5  # 임베딩 된 차원의 크기 및 RNN 층 입력 차원의 크기
hidden_size = 20  # RNN의 은닉층 크기

In [10]:
# 모델 생성
model = Net(vocab_size, input_size, hidden_size, batch_first=True)
# 손실함수 정의
loss_function = nn.CrossEntropyLoss() # 소프트맥스 함수 포함이며 실제값은 원-핫 인코딩 안 해도 됨.
# 옵티마이저 정의
# model.parameters()를 호출하면 모델의 멤버인 2개의 nn.RNN, nn.Linear 모듈의 학습 가능한 매개변수들이 포함됩니다.
optimizer = optim.Adam(params=model.parameters(), lr=1e-3)

예측값의 크기는 (6, 8)입니다. 이는 각각 (시퀀스의 길이, 은닉층의 크기)에 해당됩니다. 

In [11]:
# 임의로 예측해보기. 가중치는 전부 랜덤 초기화 된 상태이다.
output = model(X)
print(output)
print(output.shape)

tensor([[ 0.2877, -0.1071, -0.1615, -0.2051,  0.2437,  0.2587,  0.0985,  0.3278],
        [ 0.2025, -0.3684, -0.2719, -0.3540,  0.2522,  0.1556, -0.1992,  0.3091],
        [ 0.3769, -0.1722, -0.0732, -0.3362,  0.2417,  0.2056,  0.0040,  0.2915],
        [ 0.2672, -0.3560,  0.0371, -0.2804,  0.2098,  0.0690,  0.2217,  0.0878],
        [ 0.4085, -0.2855,  0.0270, -0.2451,  0.1499,  0.1609,  0.1763,  0.1656],
        [ 0.5663, -0.0194, -0.0607, -0.1999,  0.0964,  0.0972,  0.0847,  0.1650]],
       grad_fn=<ViewBackward>)
torch.Size([6, 8])


In [12]:
# 수치화된 데이터를 단어로 전환하는 함수
decode = lambda y: [index2word.get(x) for x in y]

In [13]:
# 훈련 시작
for step in range(201):
    optimizer.zero_grad()                      # 경사 초기화
    output = model(X)                          # 순방향 전파
    loss = loss_function(output, Y.view(-1))   # 손실값 계산
    loss.backward()                            # 역방향 전파
    optimizer.step()                           # 매개변수 업데이트
    
    # 기록
    if step % 40 == 0:
        print("[{:02d}/201] {:.4f} ".format(step+1, loss))
        pred = output.softmax(-1).argmax(-1).tolist()
        print(" ".join(["Repeat"] + decode(pred)))
        print()

[01/201] 2.1727 
Repeat for for <unk> <unk> <unk> <unk>

[41/201] 1.6300 
Repeat is the best best for memory

[81/201] 1.0255 
Repeat is the best medicine for memory

[121/201] 0.5727 
Repeat is the best medicine for memory

[161/201] 0.3158 
Repeat is the best medicine for memory

[201/201] 0.1879 
Repeat is the best medicine for memory

